# Getting school expenditure data from online expenditure reports

In this notebook, we scrape the online NYC school expenditure reports. Specifically, we're using the schools listed in the 2006-2012 Demographic and Accountability snapshot available on NYC Open Data.

In [1]:
from urllib2 import urlopen, HTTPError, URLError
from lxml.html import parse
import re
import pandas as pd
import numpy as np
from retrying import retry

## define helper function hexencode to handle special characters in school name
def hexencode(matchobj):
    encoded = '%' + matchobj.group(0).encode('hex')
    return encoded

def retry_if_URL_error_not_HTTP_error(exception):
    '''Return True if we should retry (in this case when it's a URLError (i.e. network is down) and not an
    HTTPError (i.e. page doesn't exist)'''
    return (isinstance(exception, URLError) and not isinstance(exception, HTTPError))
            
##Retry decorator will retry every two seconds, for up to 10 seconds, if server side error
    
@retry(retry_on_exception=retry_if_URL_error_not_HTTP_error, wait_fixed=2000, stop_max_delay=10000)
def get_school_name_from_year_and_DBN(year, DBN):

    ##build initial query using year and DBN
    years = str(year-1) + '_' + str(year)
    query1 = ("https://www.nycenet.edu/offices/d_chanc_oper/budget/exp01/y" + years +
             "/function.asp?district=All&search=" + DBN +
             "&searchgo=Search&LCMS=**&GRANT=NO&cr1=All&cr2=All&cr3=All&cr4=All&R=1&prior=search")
    
    ##get school name by searching html page returned from query- if not found, return None.
    try:
        parsed1 = parse(urlopen(query1))
        doc=parsed1.getroot()
        options = doc.findall('.//option')
        for option in options:
            ##Schools in districts 1-32
            if re.match('District', option.text_content()):
                school_name = option.text_content()
                school_name = re.sub('--','',school_name)
                school_name = re.sub('District:\s','', school_name)
                school_name = re.sub('\s','+', school_name)
                school_name = re.sub('[^A-Za-z0-9\s+.]', hexencode, school_name)
                school_name = str(DBN) + school_name
                return school_name
            
            ##Schools in district 75- citywide special education
            elif re.match('Citywide', option.text_content()):
                school_name = option.text_content()
                school_name = re.split('--',school_name)[1]
                school_name = re.sub('\s','+',school_name)
                school_name = re.sub('\.\+?','+',school_name)
                ##Note even though it's district 75, it's coded as 97 in the url
                school_name = str(DBN) + str(97) + str(school_name)
                school_name = re.sub('[^A-Za-z0-9\s+.]', hexencode, school_name)
                return school_name 
            
            ##Schools in district 79- alternative HS's
            elif re.match('Alternative HS', option.text_content()):
                school_name = option.text_content()
                school_name = re.split('--',school_name)[1]
                school_name = re.sub('\s+','+',school_name)
                school_name = str(DBN) + str(79) + str(school_name)
                school_name = re.sub('[^A-Za-z0-9\s+.]', hexencode, school_name)
                return school_name
            
        ##If not found, return None- school expenditure report not available.
        return None
            
    except HTTPError:
        return None
    
    except URLError:
        return None
    
def get_school_data(year,DBN):
    
    school_name = get_school_name_from_year_and_DBN(year, DBN)
    if school_name is None:
        return None

    ##if found, use school name to build new query to get expenditure report for year:
    years = str(year-1) + '_' + str(year)
    query2 = ("https://www.nycenet.edu/offices/d_chanc_oper/budget/exp01/y" + years +
             "/function.asp?district=All&search=" + DBN + "&LCMS=" + DBN + school_name +
             "&schoolgo=Go&GRANT=NO&cr1=All&cr2=All&cr3=All&cr4=All&R=1&prior=search")
    try:
        parsed = parse(urlopen(query2))
        doc = parsed.getroot()
        tables = doc.findall('.//table')
        i = 1
        for table in tables:
            print 'Table #', i
            rows = table.findall('.//tr')
            for row in  rows:
                elts = row.findall('.//td')
                for val in elts:
                    print val.text_content()
            print '\n\n\n'
            i+=1
    except:  
        print "Unable to get all data"

Now we find the tables we want for each year (noting the table formating is constant by school, but not by year):

In [2]:
get_school_data(2006,'M035')

Table # 1
 
  
  DOE Home Page > 
  Offices & Programs > 
  Division of Budget Operations and Review





Table # 2





Table # 3




School Year 2005-2006  
  Report By Location | District/HS Overview | Downloads




Table # 4

   
 | District (01-32,75=SPED Citwide,79=Alt HS):
  
   
 | Sch:                  
        


   School List:
 
     Search Results -- School List Citywide Sp Ed (75) --PS 035                                            
    
 Report By:  FunctionGrant
       
  
 
Total Dollars
 
Student Type
 
Instruction Level
 
DOE Account
 
Funding Source
 

       Advanced...




Table # 5
Why do costs vary?
School Based Expenditure Reports School Year 2005-2006
Citywide Sp Ed (75)        School: M035      Title 1:  Yes
PS 035                                            
 By Total Dollars 




Table # 6
 Total Enrollment: 283 
  General Ed: 0 
  Full Time Special Ed: 283 




Table # 7
  
Total 
Pct ofSchoolExp 
PerStudAmt 
Salary 
Fringe 
Salary + Fringe 
OTPS 
 I. Direc

So, for 2006 we want:
- Table 5 (for District and Title 1)
- Table 7 all rows elements 1 and 4 (category and PerStudAmt)
- Table 10 all rows elements 1 and 4 (category and PerStudAmt)
Now let's check 2012:

In [3]:
get_school_data(2012,'M015')

Table # 1

     
  
 
  DOE Home Page > 
  Offices & Programs > 
  Division of Budget Operations and Review






Table # 2





Table # 3




School Year 2011-2012  
  Report By Location | District/HS Overview | Downloads




Table # 4

 
 | District (01-32,75=SPED Citywide,79=Alt HS):
  
   
 | Sch:                  
        


   School List:
 
     Search Results -- School List District: 01--P.S. 015 Roberto Clemente
    
 Report By:  FunctionGrant
       
  
 
Total Dollars
 
Student Type
 
Instruction Level
 
DOE Account
 
Funding Source
 

       Advanced...




Table # 5
Why do costs vary?
School Based Expenditure Reports School Year 2011-2012
District: 01       School: M015      Title 1:  Yes
P.S. 015 Roberto Clemente
 By Total Dollars 




Table # 6
 Total Enrollment: 189 
  General Ed: 159 
  Full Time Special Ed: 30 




Table # 7
  
Total 
Pct ofSchoolExp 
PerStudAmt 
Salary 
Fringe 
Salary + Fringe 
OTPS 
 I. Direct Services to Schools 
 $5,187,230 
93.3% 
$27,446 
$2,863

For 2012 we also want:
- Table 5 (for District and Title 1)
- Table 7 all rows elements 1 and 4 (category and PerStudAmt)
- Table 10 all rows elements 1 and 4 (category and PerStudAmt)
Intermediate years have essentially the same expenditure report styles- thus, we proceed.

In [4]:
@retry(retry_on_exception=retry_if_URL_error_not_HTTP_error, wait_fixed=2000, stop_max_delay=10000)
def get_all_school_data(year,DBN):
    
    school_name = get_school_name_from_year_and_DBN(year, DBN)
    if school_name is None:
        return None

    ##if found, use school name to build new query to get expenditure report for year:
    years = str(year-1) + '_' + str(year)
    query2 = ("https://www.nycenet.edu/offices/d_chanc_oper/budget/exp01/y" + years +
             "/function.asp?district=All&search=" + DBN + "&LCMS=" + school_name +
             "&schoolgo=Go&GRANT=NO&cr1=All&cr2=All&cr3=All&cr4=All&R=1&prior=search")
    
    try:
        parsed = parse(urlopen(query2))
        doc = parsed.getroot()
        tables = doc.findall('.//table')
        
        ##Table 5 data
        rows = tables[4].findall('.//tr')
        elts = rows[2].findall('.//td')
        
        ## This will work for Districts 1-32, but not 75 or 79:
        if int(school_name[4:6]) in range(1,33): 
            for val in elts[0]:
                table_5_dat = val.text_content()
                table_5_dat = re.split('[\xa0]+[\s]?',table_5_dat)
                school_features = {}
                for i in table_5_dat:
                    school_features[str(re.sub('\s','_', re.split(':\s',i)[0]))]=str(re.split(':[\s]+',i)[1])
        
        ## So we use except block to catch districts 75 and 79
        else:
            for val in elts[0]:
                table_5_dat = val.text_content()
                table_5_dat = re.split('[\s]?[\xa0]+[\s]?',table_5_dat)
                school_features = {}
                school_features['District'] = str(re.sub('\s', '_', table_5_dat[0]))
                for i in table_5_dat[1:]:
                    school_features[str(re.sub('\s','_', re.split(':\s',i)[0]))]=str(re.split(':[\s]+',i)[1])
        
        ## Now we need to convert the text district names to numeric district codes:
        if school_features['District'] == 'Citywide_Sp_Ed_(75)':
            school_features['District'] = 75
        elif  school_features['District'] == 'Alternative_HS':
            school_features['District'] = 79
        
        ##Make expenditure dict:
        expenditures = {}
        
        ##Table 7 data
        rows = tables[6].findall('.//tr')
        for row in rows:
            elts = row.findall('.//td')
            for pair in zip(elts[0],elts[3]):
                key = re.sub('[\xa0]+','_', pair[0].text_content())
                key = re.sub('.*?\._','',key)
                key = re.sub('[^A-Za-z_]','',key)
                expenditures[key] = re.sub('[^0-9]','', pair[1].text_content())
                
                
        ##Table 10 data
        rows = tables[9].findall('.//tr')
        for row in rows:
            elts = row.findall('.//td')
            for pair in zip(elts[0],elts[3]):
                key = re.sub('[\xa0]+','_', pair[0].text_content())
                key = re.sub('.*?\._','',key)
                key = re.sub('[^A-Za-z_]','',key)
                expenditures[key] = re.sub('[^0-9]','', pair[1].text_content())
                                           
    ## Note- @retry wrapper will try if URLerror but not HTTPError (so bad network connection handled)
    ## If HTTPError, we want to just return- the page doesn't exisT
    
    except HTTPError:
        return
    
    ## If data not available (or not in expected format), we'll through an IndexError:
    
    except IndexError:
        return
    
    ##clean up expenditures- read in one null value that should be deleted
    
    del expenditures['_']
    
    ##join two dictionaries and return as series
    school_features.update(expenditures)
    school_features = pd.Series(school_features.values(), index=school_features.keys(), name=DBN)
    return school_features

Now let's test this function.

In [5]:
school_features_M015 = get_all_school_data(2006,'M015')
school_features_X168 = get_all_school_data(2006,'X168')
school_features_K100 = get_all_school_data(2006,'K100')

In [6]:
merged = pd.concat([school_features_M015,school_features_X168,school_features_K100], axis=1)

In [7]:
school_features_M462 = get_all_school_data(2008,'M462')
merged = merged.join(school_features_M462)

In [8]:
merged

,M015,X168,K100,M462
Instructional_Support_Srcs_All_Funds,2884,12864,1149,2687
Central_Instructional_Support_All_Funds,33,36,32,57
Attendance__Outreach_Services,43,94,163,291
District,01,75,21,06
Counseling_Services,634,349,130,571
Retiree_Health_and_Welfare,309,309,309,361
Related_Services,946,11445,193,413
_No_type_required,261,510,188,310
Instructional_Offices,69,76,67,72
Computer_System_Support_School_Level,101,98,100,122


Great! Now let's get all the data for all the years.

In [9]:
def get_DBN_list():
    query = 'https://nycopendata.socrata.com/api/views/ihfw-zy9j/rows.csv?accessType=DOWNLOAD'
    Demo_and_Account = pd.read_csv(query)
    all_DBNs = Demo_and_Account['DBN']
    unique_DBNs = all_DBNs.unique()
    for i in range(len(unique_DBNs)):
        unique_DBNs[i] = unique_DBNs[i][2:]
    return unique_DBNs

In [10]:
def build_expenditure_dataframe(year):
    unique_DBNs = get_DBN_list()
    file_path = 'school_supply_expenditures_' + str(year)
    results_for_year = get_all_school_data(year, unique_DBNs[0])
    for DBN in unique_DBNs[1:]:
        school_results = get_all_school_data(year, DBN)
        results_for_year = pd.concat([results_for_year, school_results], axis=1)
    results_for_year = results_for_year.T
    return results_for_year

In [ ]:
def save_2006_to_2012_data():
    print 'Starting to get data'
    for year in range(2006,2013):
        print 'Starting to get data for', year
        save_path = "raw_school_expenditures_by_year/year_" + str(year) + ".csv"
        year_expenditure_data = build_expenditure_dataframe(year)
        print 'Got data for year', year
        year_expenditure_data.to_csv(save_path)
        print 'Saved data for year', year
    print 'Done'

In [ ]:
save_2006_to_2012_data()